# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 1 2022 8:30AM,240897,10,MSP211803,"Methapharm, Inc.",Released
1,Jul 1 2022 8:30AM,240897,10,MSP211804,"Methapharm, Inc.",Released
2,Jul 1 2022 8:30AM,240897,10,MSP211805,"Methapharm, Inc.",Released
3,Jul 1 2022 8:30AM,240897,10,MSP211806,"Methapharm, Inc.",Released
4,Jul 1 2022 8:30AM,240897,10,MSP211807,"Methapharm, Inc.",Released
5,Jul 1 2022 8:30AM,240900,10,8214321,Eywa Pharma Inc.,Released
6,Jul 1 2022 8:30AM,240902,10,8214338,Eywa Pharma Inc.,Released
7,Jul 1 2022 8:30AM,240898,10,Enova-10234,Emerginnova,Released
8,Jul 1 2022 8:30AM,240903,10,SNAP15218379,"Snap Medical Industries, LLC",Released
9,Jul 1 2022 8:30AM,240903,10,SNAP30183757,"Snap Medical Industries, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
24,240898,Released,1
25,240900,Released,2
26,240901,Released,23
27,240902,Released,1
28,240903,Released,4


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
240898,NaN,1.0
240900,NaN,2.0
240901,NaN,23.0
240902,NaN,1.0
240903,NaN,4.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
240804,2.0,1.0
240818,1.0,0.0
240826,4.0,14.0
240831,0.0,1.0
240834,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
240804,2,1
240818,1,0
240826,4,14
240831,0,1
240834,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,240804,2,1
1,240818,1,0
2,240826,4,14
3,240831,0,1
4,240834,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,240804,2,1
1,240818,1,
2,240826,4,14
3,240831,,1
4,240834,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 1 2022 8:30AM,240897,10,"Methapharm, Inc."
5,Jul 1 2022 8:30AM,240900,10,Eywa Pharma Inc.
6,Jul 1 2022 8:30AM,240902,10,Eywa Pharma Inc.
7,Jul 1 2022 8:30AM,240898,10,Emerginnova
8,Jul 1 2022 8:30AM,240903,10,"Snap Medical Industries, LLC"
13,Jun 30 2022 3:58PM,240901,19,"AdvaGen Pharma, Ltd"
36,Jun 30 2022 3:29PM,240896,19,"NAPP Technologies, LLC"
37,Jun 30 2022 3:25PM,240894,20,Sartorius Stedim North America
38,Jun 30 2022 3:23PM,240893,19,"NAPP Technologies, LLC"
39,Jun 30 2022 3:20PM,240892,16,American International Chemical


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jul 1 2022 8:30AM,240897,10,"Methapharm, Inc.",,5
1,Jul 1 2022 8:30AM,240900,10,Eywa Pharma Inc.,,2
2,Jul 1 2022 8:30AM,240902,10,Eywa Pharma Inc.,,1
3,Jul 1 2022 8:30AM,240898,10,Emerginnova,,1
4,Jul 1 2022 8:30AM,240903,10,"Snap Medical Industries, LLC",,4
5,Jun 30 2022 3:58PM,240901,19,"AdvaGen Pharma, Ltd",,23
6,Jun 30 2022 3:29PM,240896,19,"NAPP Technologies, LLC",,1
7,Jun 30 2022 3:25PM,240894,20,Sartorius Stedim North America,,1
8,Jun 30 2022 3:23PM,240893,19,"NAPP Technologies, LLC",,1
9,Jun 30 2022 3:20PM,240892,16,American International Chemical,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 1 2022 8:30AM,240897,10,"Methapharm, Inc.",5,
1,Jul 1 2022 8:30AM,240900,10,Eywa Pharma Inc.,2,
2,Jul 1 2022 8:30AM,240902,10,Eywa Pharma Inc.,1,
3,Jul 1 2022 8:30AM,240898,10,Emerginnova,1,
4,Jul 1 2022 8:30AM,240903,10,"Snap Medical Industries, LLC",4,
5,Jun 30 2022 3:58PM,240901,19,"AdvaGen Pharma, Ltd",23,
6,Jun 30 2022 3:29PM,240896,19,"NAPP Technologies, LLC",1,
7,Jun 30 2022 3:25PM,240894,20,Sartorius Stedim North America,1,
8,Jun 30 2022 3:23PM,240893,19,"NAPP Technologies, LLC",1,
9,Jun 30 2022 3:20PM,240892,16,American International Chemical,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 1 2022 8:30AM,240897,10,"Methapharm, Inc.",5,
1,Jul 1 2022 8:30AM,240900,10,Eywa Pharma Inc.,2,
2,Jul 1 2022 8:30AM,240902,10,Eywa Pharma Inc.,1,
3,Jul 1 2022 8:30AM,240898,10,Emerginnova,1,
4,Jul 1 2022 8:30AM,240903,10,"Snap Medical Industries, LLC",4,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jul 1 2022 8:30AM,240897,10,"Methapharm, Inc.",5.0,NaN
1,Jul 1 2022 8:30AM,240900,10,Eywa Pharma Inc.,2.0,NaN
2,Jul 1 2022 8:30AM,240902,10,Eywa Pharma Inc.,1.0,NaN
3,Jul 1 2022 8:30AM,240898,10,Emerginnova,1.0,NaN
4,Jul 1 2022 8:30AM,240903,10,"Snap Medical Industries, LLC",4.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 1 2022 8:30AM,240897,10,"Methapharm, Inc.",5.0,0.0
1,Jul 1 2022 8:30AM,240900,10,Eywa Pharma Inc.,2.0,0.0
2,Jul 1 2022 8:30AM,240902,10,Eywa Pharma Inc.,1.0,0.0
3,Jul 1 2022 8:30AM,240898,10,Emerginnova,1.0,0.0
4,Jul 1 2022 8:30AM,240903,10,"Snap Medical Industries, LLC",4.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2408865,29.0,41.0
12,240880,0.0,1.0
15,240847,0.0,29.0
16,963444,3.0,1.0
19,1445207,28.0,2.0
20,722590,23.0,4.0
22,240846,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2408865,29.0,41.0
1,12,240880,0.0,1.0
2,15,240847,0.0,29.0
3,16,963444,3.0,1.0
4,19,1445207,28.0,2.0
5,20,722590,23.0,4.0
6,22,240846,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,29.0,41.0
1,12,0.0,1.0
2,15,0.0,29.0
3,16,3.0,1.0
4,19,28.0,2.0
5,20,23.0,4.0
6,22,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,29.0
1,12,Released,0.0
2,15,Released,0.0
3,16,Released,3.0
4,19,Released,28.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,22
Status,,,,,,,
Executing,41.0,1.0,29.0,1.0,2.0,4.0,0.0
Released,29.0,0.0,0.0,3.0,28.0,23.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,22
0,Executing,41.0,1.0,29.0,1.0,2.0,4.0,0.0
1,Released,29.0,0.0,0.0,3.0,28.0,23.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,22
0,Executing,41.0,1.0,29.0,1.0,2.0,4.0,0.0
1,Released,29.0,0.0,0.0,3.0,28.0,23.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()